<a href="https://colab.research.google.com/github/PietroCaforio/research-biocv-proj/blob/dev/unimodal_ct_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Train unimodal CT

In [1]:
!git clone https://github.com/PietroCaforio/research-biocv-proj
!cd research-biocv-proj && git switch dev

Cloning into 'research-biocv-proj'...
remote: Enumerating objects: 150, done.
remote: Counting objects: 100% (150/150), done.
remote: Compressing objects: 100% (115/115), done.
remote: Total 150 (delta 74), reused 93 (delta 27), pack-reused 0 (from 0)
Receiving objects: 100% (150/150), 3.37 MiB | 21.59 MiB/s, done.
Resolving deltas: 100% (74/74), done.
Branch 'dev' set up to track remote branch 'dev' from 'origin'.
Switched to a new branch 'dev'


In [2]:
!cd research-biocv-proj && git pull

Already up to date.


In [3]:
!pip install wandb

In [4]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("wandb_api_key")

In [5]:
import wandb
wandb.login(key=secret_value_0)

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [6]:
import sys
from pathlib import Path

# Add the 'data' directory to sys.path
sys.path.append(str(Path('research-biocv-proj').resolve()))
from data.unimodal import *
from pathlib import Path

import numpy as np
import torch
from torch.utils.data import DataLoader

### Train ResNet model

In [7]:
def train(model,config, run_name=None):
  wandb.init(
    # set the wandb project where this run will be logged
    project="unimodal_ct_training",
    name = run_name,
    # track hyperparameters and run metadata
    config=config
  )
  optimizer = optim.Adam(model.parameters(), lr=config["learning_rate"], weight_decay=config["weight_decay"])
  criterion = nn.CrossEntropyLoss()
  scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min',factor = config["reduce_lr_factor"], patience = config["patience"])
  # Training loop
  num_epochs = config["epochs"]
  for epoch in range(num_epochs):
      model.train()
      running_loss = 0.0

      for batch in train_loader:
          frames = batch['frame'].float().to(device)
          labels = batch['label'].long().to(device)

          optimizer.zero_grad()
          outputs = model(frames)
          loss = criterion(outputs.logits, labels)

          loss.backward()
          optimizer.step()

          running_loss += loss.item()

      print(f"Epoch {epoch+1}, Loss: {running_loss/len(train_loader)}")

      # Validation loop
      model.eval()
      val_loss = 0.0
      correct = 0
      total = 0
      # Initialize counters for each class (G1, G2, G3)
      correct_per_class = [0, 0, 0]  # For G1, G2, G3
      total_per_class = [0, 0, 0]  # For G1, G2, G3

      with torch.no_grad():
          for batch in val_loader:
              frames = batch['frame'].float().to(device)
              labels = batch['label'].long().to(device)

              outputs = model(frames)
              loss = criterion(outputs.logits, labels)

              val_loss += loss.item()
              _, predicted = torch.max(outputs.logits, 1)
              total += labels.size(0)
              correct += (predicted == labels).sum().item()

              # Calculate accuracy per class
              for i in range(3):  # We have 3 classes: G1 (0), G2 (1), G3 (2)
                  correct_per_class[i] += ((predicted == i) & (labels == i)).sum().item()
                  total_per_class[i] += (labels == i).sum().item()
      scheduler.step(val_loss)
      # Compute total accuracy and per-class accuracy
      total_accuracy = 100 * correct / total
      class_accuracy = [(100 * correct_per_class[i] / total_per_class[i]) if total_per_class[i] > 0 else 0 for i in range(3)]
      print(f"Validation Loss: {val_loss/len(val_loader)}, Total Accuracy: {total_accuracy:.2f}%")
      print(f"Accuracy per class - G1: {class_accuracy[0]:.2f}%, G2: {class_accuracy[1]:.2f}%, G3: {class_accuracy[2]:.2f}%")
      # log metrics to wandb
      wandb.log({"Total Accuracy": total_accuracy, "Validation Loss": val_loss/len(val_loader), "G1_Acc":class_accuracy[0], "G2_Acc":class_accuracy[1], "G3_Acc":class_accuracy[2]})
  wandb.finish()  

In [8]:
import torch.nn as nn
import torch.optim as optim
from transformers import ResNetForImageClassification

In [9]:
train_dataset = UnimodalCTDataset(split='train',dataset_path = "/kaggle/input/preprocessed57patientscptacpda/processed/" )
val_dataset = UnimodalCTDataset(split='val',dataset_path = "/kaggle/input/preprocessed57patientscptacpda/processed/")

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

### Resnet-50

In [10]:
model = ResNetForImageClassification.from_pretrained('microsoft/resnet-50')
model.classifier[-1] = nn.Linear(model.classifier[-1].in_features, UnimodalCTDataset.num_classes) #Adjusting the final layer to the unimodal number of classes

config.json:   0%|          | 0.00/69.6k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/102M [00:00<?, ?B/s]

In [11]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

ResNetForImageClassification(
  (resnet): ResNetModel(
    (embedder): ResNetEmbeddings(
      (embedder): ResNetConvLayer(
        (convolution): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        (normalization): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (activation): ReLU()
      )
      (pooler): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    )
    (encoder): ResNetEncoder(
      (stages): ModuleList(
        (0): ResNetStage(
          (layers): Sequential(
            (0): ResNetBottleNeckLayer(
              (shortcut): ResNetShortCut(
                (convolution): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
                (normalization): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
              )
              (layer): Sequential(
                (0): ResNetConvLayer(
                  (convolution): Conv2d(64

In [12]:
config={
    "learning_rate": 0.0001,
    "architecture": "microsoft/resnet-34",
    "epochs": 70,
    "weight_decay": 0.001,
    "reduce_lr_factor": 0.25,
    "patience": 20
    }
train(model, config, run_name = config["architecture"])

wandb: Currently logged in as: pietro-caforio (pietro-caforio-politecnico-di-milano). Use `wandb login --relogin` to force relogin
wandb: wandb version 0.17.9 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Tracking run with wandb version 0.17.7
wandb: Run data is saved locally in /kaggle/working/wandb/run-20240909_205202-glhasgts
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run microsoft/resnet-34
wandb: ⭐️ View project at https://wandb.ai/pietro-caforio-politecnico-di-milano/unimodal_ct_training
wandb: 🚀 View run at https://wandb.ai/pietro-caforio-politecnico-di-milano/unimodal_ct_training/runs/glhasgts


Epoch 1, Loss: 0.8927015474397842
Validation Loss: 35.74881998002529, Total Accuracy: 55.70%
Accuracy per class - G1: 0.00%, G2: 100.00%, G3: 0.00%
Epoch 2, Loss: 0.5482400390383315
Validation Loss: 45.94379528015852, Total Accuracy: 58.72%
Accuracy per class - G1: 0.00%, G2: 100.00%, G3: 6.82%
Epoch 3, Loss: 0.3270701949727045
Validation Loss: 55.03453190401196, Total Accuracy: 61.07%
Accuracy per class - G1: 0.00%, G2: 92.17%, G3: 21.97%
Epoch 4, Loss: 0.1918210360693605
Validation Loss: 52.09636842980981, Total Accuracy: 72.82%
Accuracy per class - G1: 0.00%, G2: 85.54%, G3: 56.82%
Epoch 5, Loss: 0.11487538061321598
Validation Loss: 60.08377483189106, Total Accuracy: 78.86%
Accuracy per class - G1: 0.00%, G2: 90.36%, G3: 64.39%
Epoch 6, Loss: 0.08040456882078353
Validation Loss: 65.58399643898011, Total Accuracy: 72.48%
Accuracy per class - G1: 0.00%, G2: 81.93%, G3: 60.61%
Epoch 7, Loss: 0.04900915800810677
Validation Loss: 93.78821336850524, Total Accuracy: 70.13%
Accuracy per cla

wandb:                                                                                
wandb: 
wandb: Run history:
wandb:          G1_Acc ▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
wandb:          G2_Acc ██▄▃▂▂▁▃▃▃▄▄▂▁▆▄▆▅▄▆▅▆▅▅▆▆▅▆▆▅▅▅▄▄▆▃▅▆▆▆
wandb:          G3_Acc ▁▂██▇██▇▇▆▆▇▆█▆▇▆█▇▇▇▆▇▇▇▆▇▇▆▇▇▇▆█▆▇█▇▆▆
wandb:  Total Accuracy ▁▂▇▇▅▅▅▆▆▅▅▇▃▅▇▆▆█▆▇▇▆▇▇▇▇▇▇▆▇▇▇▅▇▆▆█▇▆▇
wandb: Validation Loss ▁▁▁▁▂▃▄▄▅▅▅▆▄▅▅▅▆▅▄▅▅▆▆▆▇▇█▇▇▇▇▇▇█▇▇▇███
wandb: 
wandb: Run summary:
wandb:          G1_Acc 0
wandb:          G2_Acc 92.77108
wandb:          G3_Acc 46.9697
wandb:  Total Accuracy 72.48322
wandb: Validation Loss 460.81356
wandb: 
wandb: 🚀 View run microsoft/resnet-34 at: https://wandb.ai/pietro-caforio-politecnico-di-milano/unimodal_ct_training/runs/glhasgts
wandb: ⭐️ View project at: https://wandb.ai/pietro-caforio-politecnico-di-milano/unimodal_ct_training
wandb: Synced 5 W&B file(s), 0 media file(s), 0 artifact file(s) and 0 other file(s)
wandb: Find logs at: ./wandb/run-20240909_205202-glhasgts/

### Resnet-18

In [13]:
model = ResNetForImageClassification.from_pretrained('microsoft/resnet-18')
model.classifier[-1] = nn.Linear(model.classifier[-1].in_features, UnimodalCTDataset.num_classes) #Adjusting the final layer to the unimodal number of classes

config.json:   0%|          | 0.00/69.5k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/46.8M [00:00<?, ?B/s]

In [14]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

ResNetForImageClassification(
  (resnet): ResNetModel(
    (embedder): ResNetEmbeddings(
      (embedder): ResNetConvLayer(
        (convolution): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        (normalization): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (activation): ReLU()
      )
      (pooler): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    )
    (encoder): ResNetEncoder(
      (stages): ModuleList(
        (0): ResNetStage(
          (layers): Sequential(
            (0): ResNetBasicLayer(
              (shortcut): Identity()
              (layer): Sequential(
                (0): ResNetConvLayer(
                  (convolution): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
                  (normalization): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
                  (activation): ReLU()
           

In [15]:
config={
    "learning_rate": 0.0001,
    "architecture": "microsoft/resnet-18",
    "epochs": 70,
    "weight_decay": 0.0,
    "reduce_lr_factor": 0.25,
    "patience": 20
    }
train(model, config, run_name = config["architecture"])

wandb: wandb version 0.17.9 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Tracking run with wandb version 0.17.7
wandb: Run data is saved locally in /kaggle/working/wandb/run-20240909_211931-dwo9kn4f
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run microsoft/resnet-18
wandb: ⭐️ View project at https://wandb.ai/pietro-caforio-politecnico-di-milano/unimodal_ct_training
wandb: 🚀 View run at https://wandb.ai/pietro-caforio-politecnico-di-milano/unimodal_ct_training/runs/dwo9kn4f


Epoch 1, Loss: 0.155985111102768
Validation Loss: 1.4204396684654057, Total Accuracy: 66.11%
Accuracy per class - G1: 0.00%, G2: 75.30%, G3: 54.55%
Epoch 2, Loss: 0.003125725327576319
Validation Loss: 1.4987644544104115, Total Accuracy: 63.09%
Accuracy per class - G1: 0.00%, G2: 75.30%, G3: 47.73%
Epoch 3, Loss: 0.0013499375917527773
Validation Loss: 1.5817159440834074, Total Accuracy: 69.46%
Accuracy per class - G1: 0.00%, G2: 75.30%, G3: 62.12%
Epoch 4, Loss: 0.000893415938600362
Validation Loss: 1.630158076772932, Total Accuracy: 64.77%
Accuracy per class - G1: 0.00%, G2: 75.30%, G3: 51.52%
Epoch 5, Loss: 0.0008300095605217431
Validation Loss: 1.6789563245780301, Total Accuracy: 65.10%
Accuracy per class - G1: 0.00%, G2: 75.30%, G3: 52.27%
Epoch 6, Loss: 0.000578587414889775
Validation Loss: 1.7537283330137143, Total Accuracy: 64.77%
Accuracy per class - G1: 0.00%, G2: 75.30%, G3: 51.52%
Epoch 7, Loss: 0.0005096257155942923
Validation Loss: 1.626869352729409, Total Accuracy: 66.11%


wandb:                                                                                
wandb: 
wandb: Run history:
wandb:          G1_Acc ▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
wandb:          G2_Acc ▁▁▁▁▁▁▃▁▃▃▁▄▄▁▁▁▁▂▂▁▁▁▁▄▄▁▂█▇▁▅█▇▇▅▅▃▅▄▃
wandb:          G3_Acc ▆▃▅▅▆▇▄▇▂▂▇▂▂▆▇▆▇▃▃▄▇██▄▆█▇▂▂█▃▂▂▃▁▃█▂▅▇
wandb:  Total Accuracy ▄▂▃▃▄▆▄▅▁▁▆▂▂▄▅▄▆▂▂▃▆▇▆▄▆▇▇▄▄▇▃▄▃▅▁▃█▃▆▇
wandb: Validation Loss ▁▂▄▆▄▅▅▄▇▆▅▇▇▆▆▇▆█▇▇▇▇▇▅▅▆▆▇▇▅▇████▅▄▅▅▅
wandb: 
wandb: Run summary:
wandb:          G1_Acc 0
wandb:          G2_Acc 78.31325
wandb:          G3_Acc 56.81818
wandb:  Total Accuracy 68.79195
wandb: Validation Loss 1.6995
wandb: 
wandb: 🚀 View run microsoft/resnet-18 at: https://wandb.ai/pietro-caforio-politecnico-di-milano/unimodal_ct_training/runs/dwo9kn4f
wandb: ⭐️ View project at: https://wandb.ai/pietro-caforio-politecnico-di-milano/unimodal_ct_training
wandb: Synced 5 W&B file(s), 0 media file(s), 0 artifact file(s) and 0 other file(s)
wandb: Find logs at: ./wandb/run-20240909_211931-dwo9kn4f/lo

### Resnet-34

In [16]:

model = ResNetForImageClassification.from_pretrained('microsoft/resnet-34')
model.classifier[-1] = nn.Linear(model.classifier[-1].in_features, UnimodalCTDataset.num_classes) #Adjusting the final layer to the unimodal number of classes

config.json:   0%|          | 0.00/69.5k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/87.3M [00:00<?, ?B/s]

In [17]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

ResNetForImageClassification(
  (resnet): ResNetModel(
    (embedder): ResNetEmbeddings(
      (embedder): ResNetConvLayer(
        (convolution): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        (normalization): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (activation): ReLU()
      )
      (pooler): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    )
    (encoder): ResNetEncoder(
      (stages): ModuleList(
        (0): ResNetStage(
          (layers): Sequential(
            (0): ResNetBasicLayer(
              (shortcut): Identity()
              (layer): Sequential(
                (0): ResNetConvLayer(
                  (convolution): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
                  (normalization): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
                  (activation): ReLU()
           

In [18]:
config={
    "learning_rate": 0.0001,
    "architecture": "microsoft/resnet-34",
    "epochs": 70,
    "weight_decay": 0.001,
    "reduce_lr_factor": 0.25,
    "patience": 20
    }
train(model, config, run_name = config["architecture"])


wandb: wandb version 0.17.9 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Tracking run with wandb version 0.17.7
wandb: Run data is saved locally in /kaggle/working/wandb/run-20240909_213730-6kk2upiq
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run microsoft/resnet-34
wandb: ⭐️ View project at https://wandb.ai/pietro-caforio-politecnico-di-milano/unimodal_ct_training
wandb: 🚀 View run at https://wandb.ai/pietro-caforio-politecnico-di-milano/unimodal_ct_training/runs/6kk2upiq


Epoch 1, Loss: 0.0988193004382519
Validation Loss: 0.9865438512992114, Total Accuracy: 77.52%
Accuracy per class - G1: 0.00%, G2: 75.90%, G3: 79.55%
Epoch 2, Loss: 0.0025934555714516197
Validation Loss: 1.0533404141664504, Total Accuracy: 83.56%
Accuracy per class - G1: 0.00%, G2: 75.30%, G3: 93.94%
Epoch 3, Loss: 0.0019941659594809456
Validation Loss: 0.9932094764197246, Total Accuracy: 81.88%
Accuracy per class - G1: 0.00%, G2: 75.30%, G3: 90.15%
Epoch 4, Loss: 0.012483180805957839
Validation Loss: 1.1032781890593468, Total Accuracy: 56.71%
Accuracy per class - G1: 0.00%, G2: 91.57%, G3: 12.88%
Epoch 5, Loss: 0.04464848236697535
Validation Loss: 1.7109972591511906, Total Accuracy: 61.07%
Accuracy per class - G1: 0.00%, G2: 35.54%, G3: 93.18%
Epoch 6, Loss: 0.010963352676959427
Validation Loss: 1.4378183235181496, Total Accuracy: 79.53%
Accuracy per class - G1: 0.00%, G2: 75.30%, G3: 84.85%
Epoch 7, Loss: 0.001152564424021514
Validation Loss: 1.4915508776422939, Total Accuracy: 69.80%

wandb:                                                                                
wandb: 
wandb: Run history:
wandb:          G1_Acc ▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
wandb:          G2_Acc ▃▃█▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▁▁▃▃▁▃▃▃▃▃▃
wandb:          G3_Acc ▇█▁▇▅▆▆▄▄▃▃▂▃▃▃▂████▇▇▇▇▇▇▇▄▆▇▇▇▇▇▇▇▇▇▇▆
wandb:  Total Accuracy ▇█▂▇▄▆▆▃▃▃▃▁▂▂▃▁████▇▇▇▇▇▇▇▄▅▆▆▇▇▆▇▇▇▇▇▆
wandb: Validation Loss ▁▁▁▂▂▂▁▂▂▂▂▂▂▂▂▄▁▁▁▁▃▄▃▃▃▄▄▅▃▄▇▅▆█▅▅▅▄▅▃
wandb: 
wandb: Run summary:
wandb:          G1_Acc 0
wandb:          G2_Acc 75.3012
wandb:          G3_Acc 70.45455
wandb:  Total Accuracy 73.15436
wandb: Validation Loss 2.28204
wandb: 
wandb: 🚀 View run microsoft/resnet-34 at: https://wandb.ai/pietro-caforio-politecnico-di-milano/unimodal_ct_training/runs/6kk2upiq
wandb: ⭐️ View project at: https://wandb.ai/pietro-caforio-politecnico-di-milano/unimodal_ct_training
wandb: Synced 5 W&B file(s), 0 media file(s), 0 artifact file(s) and 0 other file(s)
wandb: Find logs at: ./wandb/run-20240909_213730-6kk2upiq/lo